In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input

### mixed precision Rtcores ###
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
###
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print('Not gpu found !!!')
    
    
    
DATA_DIR = 'uc_merced'
BATCH_SIZE = 256
AUTO = tf.data.experimental.AUTOTUNE
ds, ds_info = tfds.load('uc_merced',
                        with_info=True,
                        split='train',
                        data_dir=DATA_DIR)

SHUFFLE_BUFFER = int(ds_info.splits['train'].num_examples * 0.9)

ds_train, ds_valid = tfds.load('uc_merced',
                                split=[
tfds.Split.TRAIN.subsplit(tfds.percent[:90]),
tfds.Split.TRAIN.subsplit(tfds.percent[90:])],
data_dir=DATA_DIR,
as_supervised=True)



def preprocess(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
#     image = image/255
    image = tf.image.resize(image, [256,256])
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
#     image = tf.image.random_saturation(image, lower=0.95, upper=1.05)
#     image = tf.image.random_contrast(image, lower=0.95, upper=1.05)
    return image, label
def preprocessv(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
#     image = image/255
    image = tf.image.resize(image, [256,256])
    return image, label

ds_train = (ds_train
            .map(preprocess, AUTO)
            .cache()
            .shuffle(SHUFFLE_BUFFER)
            .repeat()
            # Augmentations go here .map(augment, AUTO)
            .batch(BATCH_SIZE, drop_remainder=True)
            .prefetch(AUTO))

ds_valid = (ds_valid
            .map(preprocessv, AUTO)
            .cache()
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 2060 SUPER, compute capability 7.5
1 Physical GPUs, 1 Logical GPUs


In [2]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras import regularizers

NUM_CLASSES = ds_info.features['label'].num_classes
acti ='selu'
model = keras.Sequential([    
    
    layers.Conv2D(filters=16, kernel_size=5, padding='same', activation=acti),
    layers.MaxPool2D(),
    
    layers.BatchNormalization(),
    layers.Conv2D(32, 3, padding='same', activation=acti),
    layers.Conv2D(32, 3, padding='same', activation=acti),
    layers.MaxPool2D(),
    
    layers.BatchNormalization(),
    layers.Conv2D(64, 3, padding='same', activation=acti),
    layers.Conv2D(64, 3, padding='same', activation=acti),
    layers.MaxPool2D(),
    
    layers.BatchNormalization(),
    layers.Conv2D(128, 3, padding='same', activation=acti),
    layers.Conv2D(128, 3, padding='same', activation=acti),
    layers.MaxPool2D(),
        
    layers.BatchNormalization(),
    layers.Conv2D(256, 3, padding='same', activation=acti),
    layers.Conv2D(256, 3, padding='same', activation=acti),
    layers.MaxPool2D(),

    
    layers.Flatten(),
    layers.Dense(512, activation=acti,kernel_regularizer=regularizers.L2(0.001),kernel_initializer='lecun_normal'),
    layers.AlphaDropout(0.5),
    layers.Dense(512, activation=acti,kernel_regularizer=regularizers.L2(0.001),kernel_initializer='lecun_normal'),
    layers.AlphaDropout(0.5),
    layers.Dense(512, activation=acti,kernel_regularizer=regularizers.L2(0.001),kernel_initializer='lecun_normal'),
    layers.AlphaDropout(0.5),
    layers.Dense(512, activation=acti,kernel_regularizer=regularizers.L2(0.001),kernel_initializer='lecun_normal'),
    layers.AlphaDropout(0.5),
    layers.Dense(512, activation=acti,kernel_regularizer=regularizers.L2(0.001),kernel_initializer='lecun_normal'),
    layers.AlphaDropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax',dtype='float32')
])



In [3]:
lr = 1e-2
model.compile(
    optimizer=tf.keras.optimizers.SGD(lr),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)


In [4]:
EPOCHS = 2000
STEPS_PER_EPOCH = int(ds_info.splits['train'].num_examples * 0.9) // BATCH_SIZE #0.7

early_stopping = tf.keras.callbacks.EarlyStopping(patience=7, min_delta=0.001, restore_best_weights=True)

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH)#,callbacks=[early_stopping])

model.summary()

Epoch 1/2000
7/7 [==============================] - 41s 2s/step - loss: 4.4293 - sparse_categorical_accuracy: 0.0469 - val_loss: 4.3479 - val_sparse_categorical_accuracy: 0.0952
Epoch 2/2000
7/7 [==============================] - 3s 468ms/step - loss: 4.4425 - sparse_categorical_accuracy: 0.0485 - val_loss: 4.4699 - val_sparse_categorical_accuracy: 0.0810
Epoch 3/2000
7/7 [==============================] - 3s 464ms/step - loss: 4.3895 - sparse_categorical_accuracy: 0.0473 - val_loss: 4.5976 - val_sparse_categorical_accuracy: 0.0762
Epoch 4/2000
7/7 [==============================] - 3s 467ms/step - loss: 4.4125 - sparse_categorical_accuracy: 0.0432 - val_loss: 4.8331 - val_sparse_categorical_accuracy: 0.0476
Epoch 5/2000
7/7 [==============================] - 3s 465ms/step - loss: 4.4174 - sparse_categorical_accuracy: 0.0411 - val_loss: 4.7974 - val_sparse_categorical_accuracy: 0.0571
Epoch 6/2000
7/7 [==============================] - 3s 468ms/step - loss: 4.4189 - sparse_categorical_

7/7 [==============================] - 3s 491ms/step - loss: 4.1303 - sparse_categorical_accuracy: 0.0613 - val_loss: 7.0122 - val_sparse_categorical_accuracy: 0.0333
Epoch 47/2000
7/7 [==============================] - 3s 491ms/step - loss: 4.1330 - sparse_categorical_accuracy: 0.0611 - val_loss: 7.0696 - val_sparse_categorical_accuracy: 0.0333
Epoch 48/2000
7/7 [==============================] - 3s 488ms/step - loss: 4.1278 - sparse_categorical_accuracy: 0.0489 - val_loss: 6.5549 - val_sparse_categorical_accuracy: 0.0524
Epoch 49/2000
7/7 [==============================] - 3s 487ms/step - loss: 4.1402 - sparse_categorical_accuracy: 0.0598 - val_loss: 6.6293 - val_sparse_categorical_accuracy: 0.0524
Epoch 50/2000
7/7 [==============================] - 3s 492ms/step - loss: 4.1609 - sparse_categorical_accuracy: 0.0559 - val_loss: 6.4446 - val_sparse_categorical_accuracy: 0.0571
Epoch 51/2000
7/7 [==============================] - 3s 490ms/step - loss: 4.1153 - sparse_categorical_accura

7/7 [==============================] - 3s 493ms/step - loss: 3.8636 - sparse_categorical_accuracy: 0.0841 - val_loss: 6.0614 - val_sparse_categorical_accuracy: 0.1190
Epoch 92/2000
7/7 [==============================] - 3s 495ms/step - loss: 3.8538 - sparse_categorical_accuracy: 0.0816 - val_loss: 6.9589 - val_sparse_categorical_accuracy: 0.1143
Epoch 93/2000
7/7 [==============================] - 3s 488ms/step - loss: 3.8387 - sparse_categorical_accuracy: 0.0691 - val_loss: 6.3794 - val_sparse_categorical_accuracy: 0.1095
Epoch 94/2000
7/7 [==============================] - 3s 497ms/step - loss: 3.8489 - sparse_categorical_accuracy: 0.0836 - val_loss: 6.1088 - val_sparse_categorical_accuracy: 0.1143
Epoch 95/2000
7/7 [==============================] - 3s 495ms/step - loss: 3.8640 - sparse_categorical_accuracy: 0.0830 - val_loss: 5.4243 - val_sparse_categorical_accuracy: 0.1000
Epoch 96/2000
7/7 [==============================] - 3s 497ms/step - loss: 3.8583 - sparse_categorical_accura

7/7 [==============================] - 3s 491ms/step - loss: 3.5691 - sparse_categorical_accuracy: 0.1117 - val_loss: 10.4466 - val_sparse_categorical_accuracy: 0.1048
Epoch 137/2000
7/7 [==============================] - 3s 490ms/step - loss: 3.6376 - sparse_categorical_accuracy: 0.0926 - val_loss: 10.1604 - val_sparse_categorical_accuracy: 0.1095
Epoch 138/2000
7/7 [==============================] - 3s 493ms/step - loss: 3.5584 - sparse_categorical_accuracy: 0.1002 - val_loss: 11.3206 - val_sparse_categorical_accuracy: 0.1095
Epoch 139/2000
7/7 [==============================] - 3s 497ms/step - loss: 3.5873 - sparse_categorical_accuracy: 0.1098 - val_loss: 9.5766 - val_sparse_categorical_accuracy: 0.1048
Epoch 140/2000
7/7 [==============================] - 3s 493ms/step - loss: 3.5759 - sparse_categorical_accuracy: 0.1170 - val_loss: 8.6793 - val_sparse_categorical_accuracy: 0.1000
Epoch 141/2000
7/7 [==============================] - 3s 486ms/step - loss: 3.5511 - sparse_categorica

Epoch 181/2000
7/7 [==============================] - 3s 492ms/step - loss: 3.3373 - sparse_categorical_accuracy: 0.1517 - val_loss: 10.3959 - val_sparse_categorical_accuracy: 0.1286
Epoch 182/2000
7/7 [==============================] - 3s 488ms/step - loss: 3.2629 - sparse_categorical_accuracy: 0.1518 - val_loss: 10.3004 - val_sparse_categorical_accuracy: 0.1286
Epoch 183/2000
7/7 [==============================] - 3s 493ms/step - loss: 3.3209 - sparse_categorical_accuracy: 0.1495 - val_loss: 8.9035 - val_sparse_categorical_accuracy: 0.1381
Epoch 184/2000
7/7 [==============================] - 3s 499ms/step - loss: 3.2989 - sparse_categorical_accuracy: 0.1373 - val_loss: 10.8585 - val_sparse_categorical_accuracy: 0.1238
Epoch 185/2000
7/7 [==============================] - 3s 484ms/step - loss: 3.2847 - sparse_categorical_accuracy: 0.1442 - val_loss: 10.2687 - val_sparse_categorical_accuracy: 0.1381
Epoch 186/2000
7/7 [==============================] - 3s 495ms/step - loss: 3.3057 - s

...

7/7 [==============================] - 4s 520ms/step - loss: 1.3105 - sparse_categorical_accuracy: 0.6299 - val_loss: 57.2412 - val_sparse_categorical_accuracy: 0.2000
Epoch 1784/2000
7/7 [==============================] - 4s 519ms/step - loss: 1.3289 - sparse_categorical_accuracy: 0.6151 - val_loss: 59.8604 - val_sparse_categorical_accuracy: 0.2095
Epoch 1785/2000
7/7 [==============================] - 4s 521ms/step - loss: 1.3200 - sparse_categorical_accuracy: 0.6102 - val_loss: 59.2192 - val_sparse_categorical_accuracy: 0.2429
Epoch 1786/2000
7/7 [==============================] - 4s 521ms/step - loss: 1.3144 - sparse_categorical_accuracy: 0.6458 - val_loss: 60.6861 - val_sparse_categorical_accuracy: 0.2238
Epoch 1787/2000
7/7 [==============================] - 4s 522ms/step - loss: 1.2649 - sparse_categorical_accuracy: 0.6479 - val_loss: 39.9219 - val_sparse_categorical_accuracy: 0.2238
Epoch 1788/2000
7/7 [==============================] - 4s 516ms/step - loss: 1.2909 - sparse_cat

7/7 [==============================] - 4s 507ms/step - loss: 1.2906 - sparse_categorical_accuracy: 0.6224 - val_loss: 58.3434 - val_sparse_categorical_accuracy: 0.2190
Epoch 1828/2000
7/7 [==============================] - 4s 518ms/step - loss: 1.3056 - sparse_categorical_accuracy: 0.6154 - val_loss: 52.4603 - val_sparse_categorical_accuracy: 0.2238
Epoch 1829/2000
7/7 [==============================] - 4s 511ms/step - loss: 1.3299 - sparse_categorical_accuracy: 0.6122 - val_loss: 56.8147 - val_sparse_categorical_accuracy: 0.2000
Epoch 1830/2000
7/7 [==============================] - 4s 512ms/step - loss: 1.2799 - sparse_categorical_accuracy: 0.6319 - val_loss: 50.8139 - val_sparse_categorical_accuracy: 0.2143
Epoch 1831/2000
7/7 [==============================] - 4s 529ms/step - loss: 1.3049 - sparse_categorical_accuracy: 0.6193 - val_loss: 69.3394 - val_sparse_categorical_accuracy: 0.2143
Epoch 1832/2000
7/7 [==============================] - 4s 523ms/step - loss: 1.3153 - sparse_cat

7/7 [==============================] - 4s 520ms/step - loss: 1.2604 - sparse_categorical_accuracy: 0.6245 - val_loss: 63.8280 - val_sparse_categorical_accuracy: 0.2238
Epoch 1872/2000
7/7 [==============================] - 4s 521ms/step - loss: 1.3316 - sparse_categorical_accuracy: 0.6057 - val_loss: 50.6159 - val_sparse_categorical_accuracy: 0.2429
Epoch 1873/2000
7/7 [==============================] - 4s 519ms/step - loss: 1.2623 - sparse_categorical_accuracy: 0.6303 - val_loss: 58.6694 - val_sparse_categorical_accuracy: 0.2095
Epoch 1874/2000
7/7 [==============================] - 4s 522ms/step - loss: 1.2842 - sparse_categorical_accuracy: 0.6185 - val_loss: 47.8088 - val_sparse_categorical_accuracy: 0.2000
Epoch 1875/2000
7/7 [==============================] - 4s 507ms/step - loss: 1.2861 - sparse_categorical_accuracy: 0.6314 - val_loss: 50.3055 - val_sparse_categorical_accuracy: 0.2190
Epoch 1876/2000
7/7 [==============================] - 4s 514ms/step - loss: 1.3200 - sparse_cat

7/7 [==============================] - 4s 519ms/step - loss: 1.2453 - sparse_categorical_accuracy: 0.6337 - val_loss: 53.0490 - val_sparse_categorical_accuracy: 0.2381
Epoch 1916/2000
7/7 [==============================] - 4s 521ms/step - loss: 1.2708 - sparse_categorical_accuracy: 0.6369 - val_loss: 58.8127 - val_sparse_categorical_accuracy: 0.2238
Epoch 1917/2000
7/7 [==============================] - 4s 513ms/step - loss: 1.2328 - sparse_categorical_accuracy: 0.6383 - val_loss: 64.7806 - val_sparse_categorical_accuracy: 0.2190
Epoch 1918/2000
7/7 [==============================] - 4s 519ms/step - loss: 1.2738 - sparse_categorical_accuracy: 0.6316 - val_loss: 55.2610 - val_sparse_categorical_accuracy: 0.2095
Epoch 1919/2000
7/7 [==============================] - 4s 517ms/step - loss: 1.2817 - sparse_categorical_accuracy: 0.6247 - val_loss: 59.1391 - val_sparse_categorical_accuracy: 0.2190
Epoch 1920/2000
7/7 [==============================] - 4s 508ms/step - loss: 1.2628 - sparse_cat

7/7 [==============================] - 4s 515ms/step - loss: 1.2325 - sparse_categorical_accuracy: 0.6305 - val_loss: 75.0717 - val_sparse_categorical_accuracy: 0.1857
Epoch 1960/2000
7/7 [==============================] - 4s 505ms/step - loss: 1.2803 - sparse_categorical_accuracy: 0.6278 - val_loss: 73.9565 - val_sparse_categorical_accuracy: 0.2381
Epoch 1961/2000
7/7 [==============================] - 4s 512ms/step - loss: 1.1826 - sparse_categorical_accuracy: 0.6736 - val_loss: 60.2113 - val_sparse_categorical_accuracy: 0.2429
Epoch 1962/2000
7/7 [==============================] - 4s 517ms/step - loss: 1.2326 - sparse_categorical_accuracy: 0.6511 - val_loss: 53.9993 - val_sparse_categorical_accuracy: 0.2571
Epoch 1963/2000
7/7 [==============================] - 4s 513ms/step - loss: 1.2251 - sparse_categorical_accuracy: 0.6510 - val_loss: 53.0582 - val_sparse_categorical_accuracy: 0.2476
Epoch 1964/2000
7/7 [==============================] - 4s 513ms/step - loss: 1.3053 - sparse_cat

In [5]:
model.summary()
model.save('Trained_ucmerced')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 16)      1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 16)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 16)      64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 32)        1

INFO:tensorflow:Assets written to: Trained_ucmerced/assets
